In [66]:
import sys
sys.getdefaultencoding()

'utf-8'

In [79]:
import codecs
import chardet
import re

def check_file_encoding_type(dir):
    """
    在不知道文件 编码类型的情况下，输出文件的编码类型
    """
    with open(dir,'rb') as file: #不知道文件编码格式的情况下，使用 二级制字符流(bytes) 打开文件
        line1=file.readline()
        detect = chardet.detect(line1)
        
#         print(detect)
#         print('bytes:',line1)
#         print('decoding: ',line1.decode(detect['encoding']))
        
        return detect['encoding']
    
def convert_file_encoding(dir,encoding):
    """
    把目标文件 转换为 指定的编码格式
    """
    
    org_encoding=check_file_encoding_type(dir)
    print('original encoding:',org_encoding)
    
    file = open(dir,'r',encoding=org_encoding) 
    data=file.read()
    print('unicode str: ',data) #python3 中字符串是以Unicode编码的，字符串要写入文件前要进行 编码（encode）
    
    data=data.encode(encoding) # 使用 encoding 类型对Unicode 进行编码 
    print('after encoding: ',data)
    print(chardet.detect(data))
    
    file.close()
    
    file = open(dir, 'wb')
    file.write(data) # 写入文件
    file.close()


def edit_file(dir):
    """
    读取文件的每一行，并进行修改，然后写回到同一个文件中
    """   
# 边读边写（不靠谱）
#     with open(dir,'r+',encoding=org_encoding) as file:
#         data=file.read()       
#         print(data)
#         file.write(data)

    org_encoding=check_file_encoding_type(dir)
    
    file = open(dir, 'r',encoding=org_encoding)
    alllines=file.readlines() #全部读出来，在内存中修改  # TODO：读一行 改一行 写一行（节约内存）
    print ('original: ',alllines)
    for i in range(len(alllines)):
        
        line=alllines[i]

        if re.match(r'^\n$|^\r\n$', line) is  None: #in some txt the end of the line is \n ; while others is \r\n
            line=re.sub('\n|\r\n', ' ', line) #string replace using regular
        
        line=re.sub('[^\u4e00-\u9fa5^a-z^A-Z^0-9]', '', line) # 过滤非中英文和数字的字符串

        alllines[i]=line

    print ('after: ',alllines)
    file.close()


    file = open(dir, 'w')
    
    file.writelines(alllines)#修改好后再全部写入
    file.close()
    
    
dir1 = 'tmp/中文文本(utf-8).txt'
dir2 = 'tmp/中文文本(gbk).txt'
dir3 = 'tmp/中文文本(utf-8bom).txt'

# check_file_encoding_type(dir1)
# check_file_encoding_type(dir2)       
# check_file_encoding_type(dir3)

# check_file_encoding_type('data/all_50_schemas')
# check_file_encoding_type('data/train_data.json') # 得到encoding 为 'utf-8'后，可以在打开文件时 指定编码为'utf-8' 

# convert_file_encoding(dir2,'utf-8')
# convert_file_encoding(dir2,'gbk')

# convert_file_encoding(dir3,'gbk')
# convert_file_encoding(dir2,'UTF-8-SIG')

# edit_file(dir3)


12

In [15]:
#! -*- coding:utf-8 -*-
import json
from tqdm import tqdm
import codecs

In [26]:
all_50_schemas = set()


with open('data/all_50_schemas',encoding="utf-8") as f:
    for l in tqdm(f):
        a = json.loads(l)
        all_50_schemas.add(a['predicate'])
        
id2predicate = {i:j for i,j in enumerate(all_50_schemas)}
predicate2id = {j:i for i,j in id2predicate.items()}


with codecs.open('data/all_50_schemas_me.json', 'w', encoding='utf-8') as f:
    json.dump([id2predicate, predicate2id], f, indent=4, ensure_ascii=False)








0it [00:00, ?it/s]






50it [00:00, 16586.14it/s]

In [25]:

train_data = []

chars = {}

with open('data/train_data.json',encoding="utf-8") as f:
    for l in tqdm(f):
        a = json.loads(l)
#         print(a)
        if not a['spo_list']:
            continue
        train_data.append(
            {
                'text': a['text'],
                'spo_list': [(i['subject'], i['predicate'], i['object']) for i in a['spo_list']]
            }
        )
        for c in a['text']:
            chars[c] = chars.get(c, 0) + 1


with codecs.open('data/train_data_me.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, indent=4, ensure_ascii=False)








0it [00:00, ?it/s]






1719it [00:00, 17062.77it/s]






3657it [00:00, 17665.50it/s]






6011it [00:00, 18452.80it/s]






7969it [00:00, 18686.72it/s]






10010it [00:00, 19169.82it/s]






12462it [00:00, 19751.58it/s]






14609it [00:00, 20115.20it/s]






16845it [00:00, 19922.02it/s]






18936it [00:00, 20159.94it/s]






20889it [00:01, 19388.16it/s]






22790it [00:01, 19158.48it/s]






24687it [00:01, 19100.47it/s]






26579it [00:01, 18653.97it/s]






28540it [00:01, 18929.09it/s]






30427it [00:01, 18797.79it/s]






32303it [00:01, 18399.48it/s]






34427it [00:01, 18938.43it/s]






36325it [00:01, 18894.47it/s]






38301it [00:01, 19094.31it/s]






40213it [00:02, 17194.46it/s]






41988it [00:02, 17306.42it/s]






43993it [00:02, 18012.81it/s]






45888it [00:02, 18283.60it/s]






47757it [00:02, 18349.72it/s]






49606it [00:02, 18336.21it/s]






51450it [00:02, 18094.70it/s]






53354it [00:02, 18318.16it/s]








In [27]:
min_count = 2

dev_data = []


with open('data/dev_data.json', encoding='utf-8') as f:
    for l in tqdm(f):
        a = json.loads(l)
        dev_data.append(
            {
                'text': a['text'],
                'spo_list': [(i['subject'], i['predicate'], i['object']) for i in a['spo_list']]
            }
        )
        for c in a['text']:
            chars[c] = chars.get(c, 0) + 1


with codecs.open('data/dev_data_me.json', 'w', encoding='utf-8') as f:
    json.dump(dev_data, f, indent=4, ensure_ascii=False)


with codecs.open('data/all_chars_me.json', 'w', encoding='utf-8') as f:
    chars = {i:j for i,j in chars.items() if j >= min_count}
    id2char = {i+2:j for i,j in enumerate(chars)} # padding: 0, unk: 1
    char2id = {j:i for i,j in id2char.items()}
    json.dump([id2char, char2id], f, indent=4, ensure_ascii=False)








0it [00:00, ?it/s]






2305it [00:00, 21855.85it/s]






4717it [00:00, 21588.90it/s]






6854it [00:00, 21470.54it/s]






9044it [00:00, 20689.88it/s]






11129it [00:00, 20556.74it/s]






13131it [00:00, 20352.64it/s]






15178it [00:00, 20329.32it/s]






17165it [00:00, 20187.56it/s]






19060it [00:00, 19435.86it/s]






20923it [00:01, 18951.21it/s]






21639it [00:01, 19842.09it/s]